In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [34]:
import tomllib

with open('../.config.toml', 'rb') as f:
    _CONFIGS = tomllib.load(f)

with open('../.tokens.toml', 'rb') as f:
    _TOKENS = tomllib.load(f)

In [4]:
from utils import load_test_cases

test_cases = load_test_cases('../data/test_cases.txt')
test_cases[:3]

[{'q': '什么是我国第一部编年史著作？', 'a': '《左传》。', 'docs': ['附：《左传》是我国第一部编年史著作。\n']},
 {'q': '什么是我国第一部编年国别史？', 'a': '《国语》。', 'docs': ['附：《国语》是我国第一部编年国别史。\n']},
 {'q': '“寡人之于国也”下一句是什么？来自哪里？',
  'a': '“寡人之于国也”下一句是“尽心焉耳矣”。这个句子来自《孟子》。',
  'docs': ['梁惠王曰：“寡人之于国也，尽心焉耳矣。河内凶，则移其民于河东，移其粟于河内；河东凶亦然。察邻国之政，无如寡人之用心者。邻国之民不加少，寡人之民不加多，何也？”',
   '《寡人之于国也》（孟子）\n']}]

In [5]:
from langchain_community.llms import LlamaCpp

llm = LlamaCpp(
    model_path=_CONFIGS['model_path']+'/'+_CONFIGS['model_mapping'][_CONFIGS['model_name']],
    name=_CONFIGS['model_name'], 
    **_CONFIGS['llm']
)

/Users/fred/micromamba/envs/my-notion-companion/lib/python3.12/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! conversation is not default parameter.
                conversation was transferred to model_kwargs.
                Please confirm that conversation is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /Users/fred/Documents/models/zephyr-7b-beta.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096


In [44]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    _CONFIGS["model_name"], trust_remote_code=True
)

In [45]:
import tomllib

with open("../templates/query_constructor_template.toml", "rb") as f:
    templates = tomllib.load(f)

In [46]:
from my_notion_companion.few_shot_constructor import FewShotTemplateConstructor

few_shot = FewShotTemplateConstructor(tokenizer, templates)
s = few_shot.invoke("新裤子乐队最出名的专辑是什么时候发售的？请简单介绍该乐队。")
s

'<|system|>\n你是一个乐于助人的AI助手，你的唯一任务是从给定的文本中找到关键词和搜索范围。关键词将用于后续在搜索引擎中进行搜索。搜索范围将定义搜索的数据库。关键词和搜索范围只能完全来源于文本中，禁止添加其他内容。搜索范围的默认输出为“无”，除非文本中有明确要求（如：“请从某处寻找答案”）。如果有多个关键词或搜索范围，用空格“ ”分隔。\n</s>\n<|user|>\n<< 文本 >>\n你是谁?</s>\n<|assistant|>\n<< 关键词和搜索范围 >>\n关键词：你是谁\n搜索范围：无</s>\n<|user|>\n<< 文本 >>\n小王认识小李相识吗？他们是怎么认识的？请从资料库中找到答案。</s>\n<|assistant|>\n<< 关键词和搜索范围 >>\n关键词：小王和小李\n搜索范围：资料库</s>\n<|user|>\n<< 文本 >>\n请列举一个关于“深秋初冬”的描写片段。请从“文学”中寻找。</s>\n<|assistant|>\n<< 关键词和搜索范围 >>\n关键词：深秋初冬\n搜索范围：文学</s>\n<|user|>\n<< 文本 >>\n李白针对《逍遥游》写过一首什么诗？答案在古代诗歌中。</s>\n<|assistant|>\n<< 关键词和搜索范围 >>\n关键词：李白 逍遥游\n搜索范围：古代诗歌</s>\n<|user|>\n<< 文本 >>\n“鹅鹅鹅，曲项向天歌”是哪位诗人所作？何时所作？请只给出诗人名字。</s>\n<|assistant|>\n<< 关键词和搜索范围 >>\n关键词：鹅鹅鹅，曲项向天歌\n搜索范围：无</s>\n<|user|>\n<< 文本 >>\n“结婚是想象战胜理智”下一句是什么？来自哪里？</s>\n<|assistant|>\n<< 关键词和搜索范围 >>\n关键词：结婚是想象战胜理智\\n搜索范围：无</s>\n<|user|>\n<< 文本 >>\n哪一本小说谈到了“后现代性”的话题？请根据“摘抄”回答。</s>\n<|assistant|>\n<< 关键词和搜索范围 >>\n关键词：后现代性\\n搜索范围：小说 摘抄</s>\n<|user|>\n<< 文本 >>\n“我们之所以突然变得残暴”的后面是什么？请从三岛由纪夫的小说中寻找答案。</s>\n<|ass

In [47]:
from langchain_core.runnables import RunnableLambda
c = RunnableLambda(few_shot.invoke) | llm

c.invoke("“鹅鹅鹅，曲项向天歌”是哪位诗人所作？何时所作？请只给出诗人名字。")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    8600.66 ms
llama_print_timings:      sample time =       2.46 ms /    28 runs   (    0.09 ms per token, 11372.87 tokens per second)
llama_print_timings: prompt eval time =    2832.99 ms /   559 tokens (    5.07 ms per token,   197.32 tokens per second)
llama_print_timings:        eval time =     982.06 ms /    27 runs   (   36.37 ms per token,    27.49 tokens per second)
llama_print_timings:       total time =    3862.99 ms /   586 tokens


'关键词：鹅鹅鹅，曲项向天歌\\n搜索范scope：无'

In [44]:
from my_notion_companion.query_analyzer import QueryAnalyzer

qa = QueryAnalyzer(llm, _CONFIGS)

In [50]:
for case in test_cases:
    r = qa.invoke(case['q'])
    print(case['q'])
    print(r)
    print("-"*30)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4042.14 ms
llama_print_timings:      sample time =       2.47 ms /    28 runs   (    0.09 ms per token, 11322.28 tokens per second)
llama_print_timings: prompt eval time =     450.03 ms /    45 tokens (   10.00 ms per token,    99.99 tokens per second)
llama_print_timings:        eval time =    1112.13 ms /    27 runs   (   41.19 ms per token,    24.28 tokens per second)
llama_print_timings:       total time =    1609.82 ms /    72 tokens
Llama.generate: prefix-match hit


什么是我国第一部编年史著作？
{'keywords': ['我国第一部编年史著作'], 'domain': ['国家档案局数据库']}
------------------------------



llama_print_timings:        load time =    4042.14 ms
llama_print_timings:      sample time =       2.16 ms /    25 runs   (    0.09 ms per token, 11579.43 tokens per second)
llama_print_timings: prompt eval time =     333.21 ms /    45 tokens (    7.40 ms per token,   135.05 tokens per second)
llama_print_timings:        eval time =     992.47 ms /    24 runs   (   41.35 ms per token,    24.18 tokens per second)
llama_print_timings:       total time =    1367.90 ms /    69 tokens
Llama.generate: prefix-match hit


什么是我国第一部编年国别史？
{'keywords': ['第一部编年史或编年国别史'], 'domain': ['我国']}
------------------------------



llama_print_timings:        load time =    4042.14 ms
llama_print_timings:      sample time =       6.18 ms /    70 runs   (    0.09 ms per token, 11330.53 tokens per second)
llama_print_timings: prompt eval time =     337.71 ms /    53 tokens (    6.37 ms per token,   156.94 tokens per second)
llama_print_timings:        eval time =    2881.45 ms /    69 runs   (   41.76 ms per token,    23.95 tokens per second)
llama_print_timings:       total time =    3343.68 ms /   122 tokens
Llama.generate: prefix-match hit


...

“我相信某一天，在某个人的生命中，他头顶天空中所有云朵的形状将会片刻地与他相似。”出自哪里？
{'keywords': ['我相信某一天，在某个人的生命中，他头顶天空中所有云朵的形状将会片刻地与他相似'], 'domain': ['无']}
------------------------------



llama_print_timings:        load time =    4042.14 ms
llama_print_timings:      sample time =       3.09 ms /    36 runs   (    0.09 ms per token, 11635.42 tokens per second)
llama_print_timings: prompt eval time =     506.71 ms /    78 tokens (    6.50 ms per token,   153.93 tokens per second)
llama_print_timings:        eval time =    1668.50 ms /    35 runs   (   47.67 ms per token,    20.98 tokens per second)
llama_print_timings:       total time =    2244.26 ms /   113 tokens
Llama.generate: prefix-match hit


有一个诗人曾经写过类似“有人把国家带进沟里，但这些人说统治是一门艺术”的话。请找到出处。
{'keywords': ['有人把国家带进沟里，但这些人说统治是一门艺术'], 'domain': ['无']}
------------------------------



llama_print_timings:        load time =    4042.14 ms
llama_print_timings:      sample time =       2.46 ms /    29 runs   (    0.08 ms per token, 11764.71 tokens per second)
llama_print_timings: prompt eval time =     359.47 ms /    63 tokens (    5.71 ms per token,   175.26 tokens per second)
llama_print_timings:        eval time =    1337.96 ms /    28 runs   (   47.78 ms per token,    20.93 tokens per second)
llama_print_timings:       total time =    1777.49 ms /    91 tokens


心学和理学的区别是什么？请从《中国哲学简史》中找到相关资料。
{'keywords': ['心学和理学的区别'], 'domain': ['《中国哲学简史》']}
------------------------------
